In [ ]:
import os
from pathlib import Path
from datetime import date

import gc
gc.enable()

# snappy imports
import snappy
from snappy import ProductIO, GPF

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt

In [ ]:
# scihub credentials
SCIHUB_UN = 'fillme'
SCIHUB_PW = 'fillme'

# request parameters - more TBD
REQUEST_AREA = "POLYGON ((6.2567138671875 51.57536261053028, 6.7160797119140625 51.57536261053028, 6.7160797119140625 51.78865666323309, 6.2567138671875 51.78865666323309, 6.2567138671875 51.57536261053028))"

# output directories 
TMP_OUTPUT_DIR = str(Path.home()) + "/output/tmp/"
FNL_OUTPUT_DIR = str(Path.home()) + "/output/"

sentinel_ids = "6492505e-3777-4d00-88d2-d10df3d04955" #"c001a5c2-db67-403e-9ad6-0c6e6a0d903d"

print('REQUEST_AREA:' + REQUEST_AREA)
print('sentinel_ids:' + sentinel_ids)

In [ ]:
Path(TMP_OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
Path(FNL_OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

In [ ]:
# download file (doesn't get downloaded if already exists)
api = SentinelAPI(SCIHUB_UN, SCIHUB_PW)
downloaded_file = api.download(sentinel_ids, directory_path=FNL_OUTPUT_DIR)
downloaded_file

In [ ]:
import folium
from shapely import wkt, geometry

dataset_footprint = wkt.loads(downloaded_file['footprint'])
aoi_footprint = wkt.loads(REQUEST_AREA)

m = folium.Map(zoom_starts=3)
m.fit_bounds((
    (dataset_footprint.bounds[1], dataset_footprint.bounds[0]), 
    (dataset_footprint.bounds[3], dataset_footprint.bounds[2])))
folium.GeoJson(dataset_footprint).add_to(m)
folium.GeoJson(aoi_footprint).add_to(m)
m

## Sentinel Preprocessing using Snappy

In [ ]:
import shutil
import glob

def delete_output(output_name):
    filepaths = glob.glob(output_name + "*")
    for filepath in filepaths:
        try:
            if os.path.isdir(filepath):
                shutil.rmtree(filepath, ignore_errors=True)
            else: 
                os.remove(filepath)
        except Exception as e:
            print("Error while deleting path: ", filepath, e)

# Some initial configurations
snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
sentinel_image = ProductIO.readProduct(downloaded_file['path'])
HashMap = snappy.jpy.get_type('java.util.HashMap')

In [ ]:
%%time
# Step: Apply Orbit Files
step1_output = os.path.join(TMP_OUTPUT_DIR, "step1_orbit_files") 

# parameters
parameters = HashMap()

# create and write product
step1_product = GPF.createProduct("Apply-Orbit-File", parameters, sentinel_image)
ProductIO.writeProduct(step1_product, step1_output, 'BEAM-DIMAP')  

In [ ]:
%%time
# Step: Calibration
step2_output = os.path.join(TMP_OUTPUT_DIR, "step2_calibration")

parameters = HashMap()
parameters.put('outputSigmaBand', True)
parameters.put('outputImageScaleInDb', False)

step2_product = snappy.GPF.createProduct("Calibration", parameters, ProductIO.readProduct(step1_output + ".dim"))
ProductIO.writeProduct(step2_product, step2_output, 'BEAM-DIMAP')

# free space of previous step
delete_output(step1_output)

In [ ]:
%%time
# Step: Subsetting to area of interest
step3_output = os.path.join(TMP_OUTPUT_DIR, "step3_subset")

WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')
geom = WKTReader().read(REQUEST_AREA)

parameters = HashMap()
parameters.put("geoRegion", geom)
parameters.put("outputImageScaleInDb", False)

step3_product = GPF.createProduct("Subset", parameters, ProductIO.readProduct(step2_output + ".dim"))
ProductIO.writeProduct(step3_product, step3_output, 'BEAM-DIMAP')

# free space of previous step
delete_output(step2_output)

In [ ]:
%%time
# Step: Speckle Filtering
step4_output = os.path.join(TMP_OUTPUT_DIR, "step4_speckle")

parameters = HashMap()
parameters.put("filter", "Lee")
parameters.put("filterSizeX", 5)
parameters.put("filterSizeY", 5)

step4_product = GPF.createProduct("Speckle-Filter", parameters, ProductIO.readProduct(step3_output + ".dim"))
ProductIO.writeProduct(step4_product, step4_output, "BEAM-DIMAP")

# free space of previous step
delete_output(step3_output)

In [ ]:
%%time
# Step: Terrain Correction
step5_output = os.path.join(FNL_OUTPUT_DIR, "corrected_output")

parameters = HashMap()
parameters.put('demName', 'SRTM 3Sec') 
parameters.put('pixelSpacingInMeter', 10.0) 
parameters.put('imgResamplingMethod', 'BILINEAR_INTERPOLATION') #BILINEAR_INTERPOLATION NEAREST_NEIGHBOUR
parameters.put('demResamplingMethod', 'BILINEAR_INTERPOLATION') 
# <saveDEM>false</saveDEM>

step5_product = GPF.createProduct("Terrain-Correction", parameters, ProductIO.readProduct(step4_output + ".dim")) 
ProductIO.writeProduct(step5_product, step5_output, 'GeoTIFF-BigTIFF')

# free space of previous step
delete_output(step4_output)

In [ ]:
os.remove(downloaded_file['path'])

## Classifying Water Pixels

In [ ]:
import os

import rasterio 
import rasterio.features
import rasterio.warp
from rasterio import mask
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
s1_corrected_path = step5_output + ".tif"

with rasterio.open(s1_corrected_path) as dataset:
    image = dataset.read(2)
    image = 10*np.log(image)
    image_bbox = dataset.bounds
    plt.figure(figsize=(10,15))
    plt.imshow(image, cmap='gray')
    plt.show()

In [ ]:
from skimage.filters import threshold_otsu, threshold_multiotsu

# compute an otsu threshold
p = plt.hist(image[~np.isinf(image)], bins=1000)
ot = threshold_multiotsu(image[~np.isinf(image)], classes=4, nbins=100)
ot = min(ot) # just take the minimum

print(f"Identified threshold according to Otsu: {ot}")

In [ ]:
inf_mask = np.isinf(image)
water_mask = image.copy()
water_mask[inf_mask] = ot + 1
water_mask = water_mask < ot

plt.figure(figsize=(15,20))
plt.imshow(water_mask, cmap='Blues')
plt.show()

In [ ]:
import folium

m = folium.Map(zoom_start=10)
m.fit_bounds(((image_bbox.bottom, image_bbox.left), (image_bbox.top, image_bbox.right)))

folium.raster_layers.ImageOverlay(
    image=water_mask,
    bounds=[[image_bbox.bottom, image_bbox.left], [image_bbox.top, image_bbox.right]],
    colormap=lambda x: (0, 0, x, max(-.7+x, 0))
).add_to(m)

m

In [ ]:
with rasterio.open(s1_corrected_path) as src:
    profile = src.profile 
    
    profile.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')
    
    with rasterio.open('result.tif', 'w', **profile) as dst_dataset:
        dst_dataset.write(water_mask.astype(rasterio.uint8), 1)